### **Librerías**

In [18]:
import os
import math
import time
import smtplib

import pyautogui
import threading
import pywhatkit

import numpy as np
import pandas as pd

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

InternetException: Error while connecting to the Internet. Make sure you are connected to the Internet!

### **BigQuery**

In [19]:
# Proyecto y cliente
project_id='bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Configuración job
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

query = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_backlog_tarde`
"""

# Query execution
query_job = client.query(query, job_config=job_config)
query_job.result()
data = query_job.to_dataframe()

c:\Users\gfloress\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### **Variables**

In [20]:
# Correo

correo_usuario = 'sleona@falabella.com'
contrasena = 'I$H#pm3457'
asunto_base = 'F.COM | 🚨 GESTIONA TUS ÓRDENES PENDIENTES 🚨'

# Columnas

columnas = ['sellerName', 'trackingCode','deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'Estatusenvio']

# Variables iniciales

### **Data**

In [21]:
# Leer el registro desde un archivo Excel
data = data.query("sellerEmail.str.contains('@')")

### **Condicionales**

In [22]:
# Correo Other Sellers only Pending
data_1 = (
            data
            .query('(lastStatusGSC == "pending") and not (accion in ["No accionar", "Cancelar", "Cambio de estado", "Comunicar envío día siguiente"])')
)

### **Envío de correos**

In [5]:
#BLOQUE PARA ENVIAR MAILS ONLY PENDINGS

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_1= data_1.map(lambda x: str(x))

# Función para enviar correos electrónicos
def enviar_correo(correo_usuario, contrasena, correo_destinatario, mensaje):
    try:
        # Establecer la conexión con el servidor SMTP de Outlook
        server = smtplib.SMTP('smtp.office365.com', 587, timeout=30)
        server.starttls()
        
        # Iniciar sesión en el servidor SMTP
        server.login(correo_usuario, contrasena)
        
        # Enviar el correo electrónico
        server.sendmail(correo_usuario, correo_destinatario, mensaje.as_string())
        
        print("Correo electrónico enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar el correo electrónico: {e}")

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_1['seller_ID'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_1[data_1['seller_ID'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['Correo'].iloc[0]
    seller_name = df_seller['sellerName'].iloc[0]

    # Renombrar las columnas
    df_seller = (
                  df_seller[columnas]
                  .drop_duplicates()
                  .rename(columns={
                            'seller_Name': 'Nombre del seller',
                            'deliveryOrderNumber': 'Número de orden',
                            'trackingCode': 'Rastreo',
                            'description': 'Descripción',
                            'promisedBySeller': 'Fecha de promesa',
                            'Estatusenvio': 'Estado del envío'
                  })
                )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {seller_name}, con ID {seller}! Te recordamos que algunos pedidos aún no están marcados como "Listo para despachar"!</strong></h2></p>
          <p>Por favor, confirma antes de las 10:00 pm de HOY para evitar problemas para el cliente y mantener tu rendimiento.</p>
          <p>Recordar que debe realizar este cambio para que la(s) órden(es) sean (programadas) para su recolección en la fecha límite de despacho.</p>
          <br>
          {df_seller.to_html(index=False)}
          <br>

          
          <p><strong>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</strong></p>

              
          <p>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</p>
          
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
              
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico llamando a la función definida anteriormente
    enviar_correo(correo_usuario, contrasena, correo_seller, msg)
    
server.quit()

C:\Users\darroyop\AppData\Local\Temp\ipykernel_36408\1258853510.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  info_backlog_4= info_backlog_4.applymap(lambda x: str(x))


Correo electrónico enviado exitosamente.


NameError: name 'server' is not defined